In [ ]:
# Install deps
%pip install "anthropic[vertex]"
# https://github.com/anthropics/anthropic-sdk-go

In [ ]:
from anthropic import AnthropicVertex
from os import environ
region = environ.get("CLOUD_ML_REGION", "")
project_id = environ.get("ANTHROPIC_VERTEX_PROJECT_ID", "")
client = AnthropicVertex(region=region, project_id=project_id)
model = "claude-sonnet-4-5@20250929"

In [ ]:
from typing import List, Optional

def add_user_message(messages: List[dict], content: str):
    _add_role_message(messages, "user", content)

def add_agent_message(messages: List[dict], content: str):
    _add_role_message(messages, "assistant", content)

def _add_role_message(messages: List[dict], role: str, content: str):
    messages.append({
        "role": role,
        "content": content
    })

""" Sends a chat request to the Claude model with the given messages. """
def chat(messages: List[dict], 
         system: Optional[str] = None,
         temperature: float = 1.0,
         stop_sequences: Optional[str] = None) -> str:
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences
    }
    
    if system is not None:
        params["system"] = system
    
    response = client.messages.create(**params)
    return response.content[0].text


In [ ]:
# Start with an empty message list
messages = []

# Add the initial user question
add_user_message(messages, "Define quantum computing in one sentence")

# Get Claude's response
answer = chat(messages)

# Add Claude's response to the conversation history
add_agent_message(messages, answer)

# Add a follow-up question
add_user_message(messages, "Write another sentence")

# Get the follow-up response with full context
final_answer = chat(messages)

final_answer

In [ ]:
messages: List[dict] = []

while True:
    user_input = input("You: ")
    add_user_message(messages, user_input)
    response = chat(messages)
    add_agent_message(messages, response)
    print("----------------")
    print(response)
    print("----------------")

In [ ]:
messages: List[dict] = []   

add_user_message(messages,
                  "Write a python function that checks a string for duplicate chars",)
answer = chat(messages, 
              system="You are a software engineer. Provide only compact code snippets.",
              temperature=0.2)
print("Claude:", answer)

In [ ]:
# Streaming example
messages: List[dict] = []
add_user_message(messages, "Explain the theory of relativity in simple terms.")
with client.messages.stream(
    model=model,
    max_tokens=1000,
    messages=messages
) as stream:
    for text in stream.text_stream:
        pass
        print(text, end="", flush=True)  # Send to client in real application
    
final_message = stream.get_final_message()
print(f"Final message:{final_message}")

In [ ]:
messages: List[dict] = []   

add_user_message(messages,
                  "Coffee or Tea ?")

## Example of message prefilling
add_agent_message(messages,
                  "Coffee is better because",)
answer = chat(messages)
print("Claude:", answer)

In [ ]:
messages: List[dict] = []
add_user_message(messages, "Count from 1 to 10")
answer = chat(messages, stop_sequences=["5"])
answer

In [ ]:
messages: List[dict] = []
add_user_message(messages, "Generate three different samples AWS cli commands. Each should be very short")
## Example of message prefilling
add_agent_message(messages,
                  "Here are all commands in a single block with no cmments\n```bash",)
answer = chat(messages, stop_sequences=["```"])
answer.strip()